Portions of this code are originally from https://github.com/qiskit/. Which is licensed under Apache License 2.0 http://www.apache.org/licenses/

The original code is © Copyright IBM 2017, 2019.

This code is licensed under the Apache License, Version 2.0. You may obtain a copy of this license in the LICENSE.txt file in the root directory of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.

Any modifications or derivative works of this code must retain this copyright notice, and modified files need to carry a notice indicating that they have been altered from the originals.

Major modifications have been made to the original code. The modifications were not made by IBM.


# Integer Factorization in N+1 Qubits

The following is a simplification of Shor's algorithm using N+1 qubits and only one application of modular exponentiation.  The drastically reduced gate count should allow it to run on existing hardware and there is no need for error correcting qubits for moderately reliable qubits because it handles error correction with redundancy.   

Shor's Original and Beauregards make up the base of the quantum function.  A simplification of only handling 2 as a base was implemnted to reduce the number of qubits.  This means that some numbers cannot be factored -- the same numbers Shor's Original cannot factor using only a base of 2. 

The key to the algorithm is that  $x^2 mod N$ as a quantum function creates a permutation matrix.  This matrix has eigenvalues, and one of those eigenvalues is the period of the function.  We do not need the exact period, but only a multiple of one of the sub-periods.  In Shor's Original, the post processing divides by 2 to get the sub-period that is needed.  In fact, Shor's goes further than necessary by finding the conjunction of both sub-periods.  Finding both sub-periods is unnecessary, as finding one sub-period provides one factor to the number.  From there, the other factor(s) are trivial. This routine attempts to access the sub-periods directly, as the sub-periods are always shorter than then full period. 

A periodic eigenvalue repeats the same vector each number of times in its period.  Here, the eigenvalues are roots of unity (that is, they are complex numbers that when raised to a power, equal 1).  Those eigenvalues must be physically represented by the qubits.  Therefore, when all the qubits return to their original positions, you know that is a multiple of a period of an eigenvalue.  Which multiple is irrelevant as the classical post processing efficiently handles even extremely large multiples. 

Because the function is periodic, it does not matter whether you find the first value that will factor the number or the 1000th value.  There is no need to worry about coordination or correlation of the qubits, as they are periodic functions.  Shor's Original spends most of its processing time eliminating these unnecessary periods.  Such calculations are faster to perform in classical post processing with the power mod function.  Even the classical post-processing function below can be significantly optimized, but there is little reason as it runs fast enough as it is, and the inefficiencies are left in so that it is easier to see how it works. 

The advantage to this method is that error correction is handled by redundancy. The gate count can be significantly reduced by switching to Big Endian  (Qiskit) format.  I leave it in this form as it is easier to understand how it works using the "textbook" version of the algorithms.  But all the swap gates can be removed from the QFT if it is done as Big Endian. 

A logical question is whether this entire routine can be efficiently classically simulated -- to which I have an answer, but it is too large to fit in the margins of this notebook. 

As an additional practical note, the classical post processing works best on numbers that are the products of two prime numbers. For the product of more than two primes, it very quickly finds the full major period which prevents it from finding non-trivial factors. However, with the full period, it is trivial to factor the number, but the classical post processing below does not do this as it adds little explanatory value.    

See Shor's initial article https://arxiv.org/abs/quant-ph/9508027v2 and its summary https://arxiv.org/abs/1301.7007v1 by John A. Smolin, Graeme Smith and Alex Vargo.  Basic outline from Stephane Beauregard (https://arxiv.org/abs/quant-ph/0205095 ).  Implemention of some functions modified from Qiskit, which appears to have borrowed from a project from TU-Delft. (https://www.tudelft.nl/en/). 

In [1]:
# You need the learn_quantum.py library which is in the same directory.
import numpy as np

from math import ceil, sqrt, log2

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
import learn_quantum as lq
from math import acos
from numpy import pi, cos, sin, exp, gcd, lcm, pi as π
from IPython.display import Latex, display
from qiskit.aqua.utils.circuit_utils import summarize_circuits

# Quantum Routines 

In [2]:
def textbook_qft(qr, start_at=0, no_swaps=False, bit_size=0):
    if bit_size == 0:
        bit_size = len(qr)
    circuit = QuantumCircuit(qr)
    
    for j in range(bit_size):
        circuit.h(qr[j+start_at])
        for k in range(1, bit_size-j):
            angle = π/(2**k)
            circuit.cu1(angle, qr[k+j+start_at], qr[j+start_at])
    
    if not no_swaps:
        for i in range(bit_size//2):
            circuit.swap(qr[i+start_at], qr[bit_size-i-1+start_at])
    
    return circuit

In [3]:
def add_integer(circuit, qr, value):
    q_count = len(qr)
    angle = value * π * (1/(2**(q_count-1)))
    for k in range(q_count):
        circuit.u1((2**k) * angle, qr[q_count - 1 - k] )


def c_add_integer(circuit, qr, control, value):
    q_count = len(qr)
    angle = value * π * (1/(2**(q_count-1)))
    # the "control" qubit can be in either location
    for k in range(q_count):
        circuit.cu1((2**k) * angle, qr[q_count - 1 - k], control )
        
        
def multiply_2_mod(circuit, qr, qr_anc, mod):
    # shift all bits left 1 <<
    shift = QuantumCircuit(qr, qr_anc)
    size = len(qr)
    for k in range(size-1):
        shift.swap(qr[k], qr[k+1])
    circuit += shift
    
    # subtract the mod
    qft = textbook_qft(qr)
    circuit += qft
    add_mod = QuantumCircuit(qr, qr_anc)
    add_integer(add_mod, qr, mod)
    circuit += add_mod.inverse()
    
    # check whether the top bit is 1 (negative)
    circuit += qft.inverse()
    circuit.cx(qr[0], qr_anc[0])
    circuit += qft
    
    # add the mod back if it was negative
    c_add_mod = QuantumCircuit(qr, qr_anc)
    c_add_integer(c_add_mod, qr, qr_anc[0], mod)
    circuit += c_add_mod
   
    #un-compute the ancillary
    circuit += qft.inverse()
    circuit += shift.inverse()
    circuit += add_mod
    circuit.x(qr[0])
    circuit.cx(qr[0], qr_anc[0])
    circuit.x(qr[0])
    circuit += add_mod.inverse()
    circuit += shift
    
    return circuit

# Classical Post-Processing Routines

In [4]:
def period_from_probability(prob):
    radians = acos(sqrt(prob))
    return period_from_radian(radians)

def period_from_radian(rad):
    #  need to adjust these number to factor larger numbers
    # these values should work for most numbers runnable on current hardware
    for k in range(1, 5000000):
        if ((k*rad) % (2*π)) < 0.00001:
            return k
    return -1

def search_answers(ones, shots, nilf, bit_size, precission):
    product = 1
    for val in ones:
        prob = round(val/shots, precission)

        per = period_from_probability(prob)
        if per > -1 :
            product *= per
        print('Period:', per)
        p=pow(2, product, nilf)

        g1 = gcd(p-1, nilf)
        if g1 > 1 and g1 < nilf:
            return g1
        g2 = gcd(p+1, nilf)
        if g2 > 1 and g2 < nilf:
            return g2
    return -1

# Quantum Function

In [11]:
# number I'd like to factor
#nilf =  34889 #123 #50621#55687#47897 # examples to test
#nilf = 376489  # Can do 20 bit numbers
nilf = 59989  ## verify it is more efficient than traditional-classical by using less than 29750 shots

bit_size = len('{0:b}'.format(nilf)) + 1 
print('Base size: ', bit_size)

# uses an ancillary register size 1 for computing mod
qr = QuantumRegister(bit_size)
qr_anc = QuantumRegister(1)
cr = ClassicalRegister(bit_size)

init = QuantumCircuit(qr, qr_anc)
init.x(qr[bit_size - 1])

final_h = QuantumCircuit(qr, qr_anc)
final_h.h(qr)

qc_multi_2 = QuantumCircuit(qr, qr_anc)
qc_multi_2 = multiply_2_mod(qc_multi_2, qr, qr_anc, nilf)

qft = textbook_qft(qr)

m = QuantumCircuit(qr, qr_anc, cr)
m.measure(qr, cr)

# increasing the shots greatly increases the probability to find
# For numbers less than 15 bits, 1000*bit_size is usually sufficient
shots = 5000*bit_size

shots = 29000 ## remove this for general testing.  

answers = lq.execute_simulated(init + qc_multi_2  + qft + final_h
                               +m, shots)


print('Done')

Base size:  17
Done


In [12]:
print(summarize_circuits(init + qc_multi_2  + qft + final_h))

Submitting 1 circuits.
0-th circuit: 18 qubits, 0 classical bits and 943 operations with depth 198
op_counts: OrderedDict([('cu1', 697), ('h', 102), ('swap', 88), ('u1', 51), ('x', 3), ('cx', 2)])



# Classical Post-Processing

The results of each qubit are combined into an array that has the number of zeros and ones for each individual qubit.  That results is used to calculate the angle of the individual qubit.  The angle is used to calculate a period.  For instance $\pi/4$ has a period of 8 -- that is, it will reach a multiple of $2\pi$ every 8 cycles. 

The periods are multiplied together to test for the sub-period. A sub-period is the number of times a factor has a greatest common divisor with a $2^x mod N$.  In Shor's Original, the sub-period is found at the end by dividing the full period by 2.  However, additional sub-periods exist, and it is always the case that one sub-period is smaller than the other.   I use the term "full period" to mean $2^x mod N = 1$.  And a sub-period is $gcd(2^x mod N, N) =$ non-trivial factor.

In [13]:
# puts the results into totals by qubit
zeros, ones = lq.results_by_qubit(answers)

# increase the precission until it is found
for precission in range(3, 7):
    factor = search_answers(ones, shots, nilf, bit_size, precission)
    if factor > -1:
        print('Found it:', factor, nilf//factor)
        break
        

Period: 4
Period: 1
Period: 1339493
Period: 492745
Period: 462635
Found it: 251 239


If it didn't find it.  Just run the quatum circuit again -- not just the classical post-processing. You can also increase he number of "shots".  As an improvement, the results from multiple smaller shot runs could be combined such that the more you ran it, the more accurate it would be. 

# How it works



In [246]:
# number I'd like to factor
nilf = 21

bit_size = len('{0:b}'.format(nilf)) + 1 
print('Base size: ', bit_size)

qr = QuantumRegister(bit_size)
qr_anc = QuantumRegister(1)

qc_multi_2_small = QuantumCircuit(qr, qr_anc)
qc_multi_2_small = multiply_2_mod(qc_multi_2_small, qr, qr_anc, nilf)

Base size:  6


In [247]:
if len(qc_multi_2_small.qregs[0]) < 8:
    lq.show_cycles(qc_multi_2_small, min_size=2)
else:
    print('Circuit too large')

<IPython.core.display.Latex object>

The period is represented inside the quantum function -- which is represented by the trasformation matrix.  It is a permutation matrix, and the "show cycles" function maps out the permutations.  

For instance, the period $2 \mapsto 4 \mapsto 8 \mapsto 16 \mapsto 32 \mapsto 22$ corresponds with the general period.  As a note, these values are pow(2, k, nilf\*2).  But they have the same period as Shor's Original.  That is, simply multiplying a number by 2 does not increase its period. It may not be that specific order is used, but one of the 6 cycle terms is used.  As a practical matter, you can always spot the size of the full period because there will be a pow(2, k, nilf\*2) period. 

In [248]:
# the matrix is too large to show, this truncates it
# and shows just a small section.  
# All the computational rows are "one-hot", even though the 1 does not appear in the truncated version
lq.show_me_the_matrix(qc_multi_2_small)

<IPython.core.display.Latex object>

The periods that correspond to the eigenvalues are in the phases:

In [249]:
if len(qc_multi_2_small.qregs[0]) < 8:
    lq.show_bloch_angles(qc_multi_2_small)
else:
    print('Circuit too large')

<IPython.core.display.Latex object>

Extract the exact phases and you can compute the full period.  This is period Shor's Original finds.  If you test the gcd after each multiple, it will find the gcd sub-periods. 

In [256]:
circuit_angles = lq.get_bloch_angles(qc_multi_2_small)
product = 1
for angles in circuit_angles:
    p=period_from_radian(round( angles[1], 10))
    product *= p

p=pow(2, product, nilf)

while p == 1:
    product = product // 2
    p = pow(2, product, nilf)

g1 = gcd(p-1, nilf)
if g1 > 1 and g1 < nilf:
    print('Found it:', g1, nilf//g1)
    
g2 = gcd(p+1, nilf)
if g2 > 1 and g2 < nilf:
    print('Found it:', g2, nilf//g2)
        

Found it: 7 3
Found it: 3 7


On a real quantum computer, we can't access those angles directly.  So, shift to frequency space with a QFT and then measure out the individual qubits in the +/- basis (perform a Hadamard on each qubit).  Measuring the results approximates the angle -- which gives us the period.

# Key Points

The key to this algorithm is the eigenvalues created by the permutation matrix.  It should be noted that a Hadamard is not used as input, therefore it is not "trying all possibilities."  Hopefully, this demonstration finally puts that notion to rest.  The secret to Shor's algorithm is math -- not magic.

This point is further illustrated by the nilf 59989 which has a period of 29750.  It can be factored in less than 29750 runs through the system, so it is clear that it could not possibly be trying every possible solution.  By the traditinal classical method, I am referring to using the power mod function to find the period.  Obviously, a number seive could be used to factor much faster for such a small number.  

# Factoring larger numbers

Several improvements could be made for factoring larger numbers in excess of 512 bits.  However, this example is more for learning how the algorithm works rather than providing an exact mechanism.

In [275]:
import qiskit.tools.jupyter
%qiskit_version_table